In [1]:
import sys
import os

package_path="C:\\Users\\Olej0\\OneDrive\\Dokumenter\\TFT_Model\\pytorch-forecasting"



# Check if the directory exists
if os.path.exists(package_path):
    # Add the package path to sys.path
    sys.path.append(package_path)


print(sys.path)

import pandas as pd
import torch
import lightning.pytorch as pl
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters



['C:\\Users\\Olej0\\OneDrive\\Dokumenter\\TFT_Model', 'C:\\Python311\\python311.zip', 'C:\\Python311\\DLLs', 'C:\\Python311\\Lib', 'C:\\Python311', 'C:\\Users\\Olej0\\OneDrive\\Dokumenter\\TFT_Model\\pytorch-forecasting\\myenv', '', 'C:\\Users\\Olej0\\OneDrive\\Dokumenter\\TFT_Model\\pytorch-forecasting\\myenv\\Lib\\site-packages', 'C:\\Users\\Olej0\\OneDrive\\Dokumenter\\TFT_Model\\pytorch-forecasting\\myenv\\Lib\\site-packages\\win32', 'C:\\Users\\Olej0\\OneDrive\\Dokumenter\\TFT_Model\\pytorch-forecasting\\myenv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\Olej0\\OneDrive\\Dokumenter\\TFT_Model\\pytorch-forecasting\\myenv\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\Olej0\\OneDrive\\Dokumenter\\TFT_Model\\pytorch-forecasting']


C:\Users\Olej0\OneDrive\Dokumenter\TFT_Model\pytorch-forecasting\pytorch_forecasting\models\base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import numpy as np
df=pd.read_csv("Final_Dataset.csv").iloc[:,1:]

"""Energy_df=pd.read_csv("FINAL_ENERGY_PRODUCTION_DAILY").iloc[:,1:]
Electricity_df=pd.read_csv("FINAL_electricity_spot").iloc[:,1:]
Electricity_df"""
targets=df.iloc[:,-3:]
df=df.iloc[:,0:-7]
print(df)


print(targets["Gas_Price_EUR"].shift(-7).values[0:30])
df["Target"]=targets["Gas_Price_EUR"]
df=df.dropna()

print(df)

lags = [30,60,100] # Add lag variables for the previous 3 time steps

# Add lag variables to the DataFrame
for column in ['EU_clean_spark_spread',"Weighted_Average_Temperature","Net_Injection","Oil_Price","Wind_energy","Electricity_price","PINDUINDEXM",  "Google_data"]:
    for lag in lags:
        df[f'{column}_EMA{lag}'] = df[column]/df[column].ewm(span=lag, adjust=False).mean().values

merged_df = df

print(merged_df)
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['month'] = merged_df['Date'].dt.month.astype(str)
merged_df['day_of_week'] = merged_df['Date'].dt.dayofweek.astype(str)
merged_df['Year'] = merged_df['Date'].dt.year.astype(str)
merged_df['Quarter'] = merged_df['Date'].dt.quarter.astype(str)
merged_df["time_idx"] = merged_df.index.tolist()
column_to_move = merged_df['time_idx']
merged_df = merged_df.drop('time_idx', axis=1)

# Step 3: Reinsert the column at the beginning
merged_df.insert(1, 'time_idx', column_to_move)
data=merged_df
data["constant"]=42



data=data.dropna()
print(len(np.array(data.columns[2:-5])[0]))

a1=np.array(data.columns[2:-5])
a2=np.array(data.columns[17:19])
print(a2)


cols=[]
for i in range(len(a1)):
  cols.append(a1[i])
  data[a1[i]] =pd.to_numeric(data[a1[i]])

cols1=[]
for i in range(len(a2)):
  cols1.append(a2[i])
  data[a2[i]] =pd.to_numeric(data[a2[i]])

cols1.append("time_idx")
cols1.append("Weighted_Average_Temperature")



cols = [item for item in cols if item not in cols1]


print(cols)
print(cols1)

            Date  Electricity_price  Exchange  CO2_spot  EU_spark_spread  \
0     2011-01-02          43.161250   1.33795    14.660       -23.613702   
1     2011-01-03          59.859583   1.33540    14.660        -6.788102   
2     2011-01-04          57.637917   1.33035    14.315        -4.858477   
3     2011-01-05          55.076250   1.31515    13.970        -5.849486   
4     2011-01-06          47.282083   1.30015    13.970       -13.583885   
...          ...                ...       ...       ...              ...   
4628  2023-09-04         108.445417   1.07960    84.000        32.398771   
4629  2023-09-05         103.542083   1.07220    84.000        28.306946   
4630  2023-09-06         105.617917   1.07275    84.000        33.918564   
4631  2023-09-07         100.985833   1.07010    84.000        30.772637   
4632  2023-09-08         108.227083   1.07005    83.045        30.899473   

      EU_clean_spark_spread  Oil_Price  Oil_Volume  Sun_energy  Wind_energy  \
0       

In [4]:
def make_dataloaders(lookback,data,pred):
    max_prediction_length = pred
    max_encoder_length = lookback
    training_cutoff = data["time_idx"].max() - max_prediction_length

    lags={
        "Target":[1,2,3,4,5,6,7,8,9,10,28,29,30,31,60],


    }

    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= training_cutoff],
        time_idx="time_idx",
        target="Target",
        group_ids=["constant"],  # keep encoder length long (as it is in the validation set)
        max_encoder_length=max_encoder_length,


        min_prediction_length=1,
        max_prediction_length=max_prediction_length,
        time_varying_known_categoricals=["day_of_week", "month","Year","Quarter"],
        time_varying_known_reals=cols1,
        time_varying_unknown_reals=cols,
        lags=lags,
        target_normalizer=GroupNormalizer(
            groups=["constant"], transformation="softplus"
        ),

        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,





    )

    validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

    batch_size = 32  # set this between 32 to 128
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=8,)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=8)
    data_df = training.data

    return train_dataloader,val_dataloader,training




In [5]:
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
def Train_model(epochs, trainer_data,test, dataset):
 

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=epochs,
        enable_model_summary=True,
        gradient_clip_val=14,
        callbacks=[lr_logger, early_stop_callback],
        logger=logger,
    )

    tft = TemporalFusionTransformer.from_dataset(
        dataset,
        learning_rate=0.01,
        hidden_size=32,
        attention_head_size=4,
        dropout=0.23,
        hidden_continuous_size=16,
        loss=QuantileLoss(),
        log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        optimizer="Ranger",
        reduce_on_plateau_patience=4,
        lstm_layers=2
    )



    trainer.fit(
        tft,
        train_dataloaders=trainer_data,
        val_dataloaders=test
 
    )
    return tft

In [ ]:
starting_point=380
Lookback=60
Len_pred=7
epoch=10


for i in range(1):
    Training_data=data.iloc[0:(starting_point+Lookback+Len_pred+i)]
    print(Training_data)
    train,test,dataset=make_dataloaders(Lookback,Training_data,Len_pred)
    TFT_Model=Train_model(epoch,train,test,dataset)
    raw_predictions =TFT_Model.predict(test, mode="raw", return_x=True)
    print(raw_predictions)
    
    
    




          Date  time_idx  Electricity_price  Exchange  CO2_spot  \
0   2011-01-02         0          43.161250   1.33795    14.660   
1   2011-01-03         1          59.859583   1.33540    14.660   
2   2011-01-04         2          57.637917   1.33035    14.315   
3   2011-01-05         3          55.076250   1.31515    13.970   
4   2011-01-06         4          47.282083   1.30015    13.970   
..         ...       ...                ...       ...       ...   
442 2012-03-19       442          46.020417   1.32415     7.650   
443 2012-03-20       443          43.437083   1.32250     7.450   
444 2012-03-21       444          45.562500   1.32120     7.250   
445 2012-03-22       445          45.346667   1.31980     7.250   
446 2012-03-23       446          41.747083   1.32730     7.250   

     EU_spark_spread  EU_clean_spark_spread  Oil_Price  Oil_Volume  \
0         -23.613702             -26.545702      94.75       71719   
1          -6.788102              -9.720102      94.84 

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Olej0\OneDrive\Dokumenter\TFT_Model\pytorch-forecasting\myenv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:197: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
C:\Users\Olej0\OneDrive\Dokumenter\TFT_Model\pytorch-forecasting\myenv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:197: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(

   | Name                               | Type                            | Params
---------------------------------------------------------------------

C:\Users\Olej0\OneDrive\Dokumenter\TFT_Model\pytorch-forecasting\myenv\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:281: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:31<00:00,  3.13s/it, v_num=42, train_loss_step=1.990]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:29<00:00,  2.96s/it, v_num=42, train_loss_step=1.130, val_loss=0.625, train_loss_epoch=2.190]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:27<00:00,  2.77s/it, v_num=42, train_loss_step=0.855, val_loss=0.569, train_loss_epoch=1.490]
Validation: 0it [00:00, ?it/s]